In [1]:
import sys
sys.path.append('../../lib/python3.5/site-packages')
import re
import jieba.analyse
import numpy as np
import jieba
jieba.set_dictionary('dict.txt.big')
jieba.analyse.set_stop_words("./ch_dont_want_word")

In [2]:
from elasticsearch import Elasticsearch
import elasticsearch.helpers
from datetime import datetime

es = Elasticsearch([{'host': '192.168.2.10', 'port': 9200}])

In [3]:
# es.get(index="pixnet", doc_type='food', id=2)

search_body = \
{\
  "_source": ["title","content","tags"], \
  "query" : {\
    "bool": {\
      "must": [\
        { \
          "match" : { \
            "content" : "台中"\
          }\
        },\
        { \
          "match" : { \
            "content" : "早午餐"\
          }\
        },\
#         { \
#           "match" : { \
#             "tags" : "夜"\
#           }\
#         }\
      ],\
#       "filter": \
#         [\
#           {\
#             "term": {\
#               "tags.keyword": {\
#                 "value": "新竹"\
#               }\
#             }\
#           },\
#           {\
#             "term": {\
#               "tags.keyword": {\
#                 "value": "宵夜"\
#               }\
#             }\
#           }\
#         ]\
    }\
  }\
}
# Take top 10
res = es.search(index="pixnet", doc_type="food", body= search_body)
print("%d documents found" % res['hits']['total'])
# for i, match in enumerate(res['hits']['hits']):
#     print("Top.{} Match\nScore : {}\nTitle : {}\nContent : {}\n\n".format(i, match['_score'], match['_source']['title'], match['_source']['content']))
#     break

1094 documents found


In [4]:
# Take all result
docs = list(elasticsearch.helpers.scan(es, index="pixnet", doc_type='food'))
total_content = [(doc['_id'],doc['_source'].get('content')) for doc in docs]
print(len(total_content))

9597


## Query Emotion

In [5]:
import urllib.request
import json

In [6]:
texts_list = total_content[0][1].split('。')

In [7]:
query = {"data":[]}
for text in texts_list:
    query["data"].append({"message":text})
query

{'data': [{'message': '說起最近高雄地區新開火紅的早午餐店，這間沐濛一定榜上有名，因為餐點份量大、價格實惠，十分受到好評'},
  {'message': '之前有在一些文章提過，高雄知名的早午餐店我也不是不想吃，只是一週唯一只有禮拜天可以睡到自然醒的我，每次睡醒加上懶懶的東摸西摸一下，可以出門的時間都已經接近中午了，而且也沒動力大老遠跑去一些大排長龍的名店朝聖，對我來說週末就是要懶散到底，別說還要訂位趕時間之類的，饒了我吧！\xa0所以週末比較常就近在鳳山隨便吃一吃，不然就是去我最愛的故事鍋物館吃涮涮鍋'},
  {'message': '這回發現沐濛離鳳山不遠ㄟ，其實跟常跑的文山特區距離差不多而已，好吧，不吃則已，要吃我就來衝第一吧！就這樣，週末的營業時間前，我們在店門口出現囉！（一定要這麼極端嗎？\xa0）時間到，沐濛準時開店囉～！上頭招牌好大片的木板上，只有小小的店名，很特別的設計，店外地板則是黑色鐵板，偏向工業風，黑色窗框外觀沉穩又時尚'},
  {'message': '裡面座位不多，還好有跑第一名，不然就沒辦法拍到這畫面啦！這家店最讓我喜歡的，是磚牆上這些畫框和鏡子，畫框裡有的沒有畫，選擇留白，鏡子裡的倒影，真真假假，虛幻難辨'},
  {'message': '\xa0因為是第一組到的客人，第一位店員對我們十分禮貌殷勤，馬上送上菜單～菜單設計的質感不錯，特別訂製的硬皮'},
  {'message': '以下菜單是從 Moment 沐濛 的官方粉絲團借圖來的～（※後來寫文的時候發現，我們現場看到的菜單品項，和臉書上菜單的版本有一點點不一樣，菜色有增減，請以現場店家提供為主，本菜單僅供參考※）點這一頁早午餐組合是最划算的，都有附飲料喝到飽（冰熱紅茶、熱美式咖啡、經典早餐茶）'},
  {'message': '江湖上（？）傳言捲餅很好吃，但我們還沒吃過'},
  {'message': '這一頁蒜辣炒小管很吸引我，但是他們家餐點份量頗大，通常一個人一個餐就能吃飽，還沒機會點到這邊來'},
  {'message': '義大利麵最貴不超過＄200，只是選擇不多就是'},
  {'message': '吸引我們來的重要理由之一，就是有美式漢堡'},
  {'message': '兒童餐，就是份量小一點，還有用可愛的卡通餐具'},
  

In [8]:
header={'Content-Type': 'application/json'}
req = urllib.request.Request(url='http://192.168.2.100:5678/chuck/couple_all', headers=header, method='POST')
query_str = json.dumps(query)


In [9]:
res = urllib.request.urlopen(req, query_str.encode())
res_json = json.loads(res.read().decode())

In [10]:
emotion_dict = {'wow':{'count':0, 'content':[]},
                'love':{'count':0, 'content':[]},
                'haha':{'count':0, 'content':[]},
                'sad':{'count':0, 'content':[]},
                'angry':{'count':0, 'content':[]}}
for sentence_res in res_json['data']:
    if sentence_res['ambiguous']!= True:
        if sentence_res['emotion1'] == 'angry':
            if sentence_res['emotion2'] == 'haha' or sentence_res['emotion2'] == 'love': continue
            
        emotion_dict[sentence_res['emotion1']]['count'] = emotion_dict[sentence_res['emotion1']]['count'] +1
        emotion_dict[sentence_res['emotion1']]['content'].append(sentence_res['message'])
emotion_dict

{'angry': {'content': ['這一頁蒜辣炒小管很吸引我，但是他們家餐點份量頗大，通常一個人一個餐就能吃飽，還沒機會點到這邊來',
   '吸引我們來的重要理由之一，就是有美式漢堡',
   '一點了餐，店員馬上給我們一張紙條，上面是手寫的WiFi密碼，也太貼心了吧！！！\xa0但是這僅只有第一次一早到訪，我們觀察過，後來的其他桌客人，或者我們第二次再訪，別的服務人員就沒有這麼積極主動貼心囉！包括其他方面的服務，像給飲料、倒茶也是，不是每個人員都很積極',
   '首先會送上檸檬水，這裡的檸檬水，可不是印象中的淡如水，是真的濃到會酸的程度，又冰涼，如果喜歡檸檬水的人，有這個其實不太需要點飲料了我覺得',
   '\xa0側面可以看到清楚的層次，土司口感好，黃色的蛋皮、紅色的番茄、綠色的生菜，還有多汁滑嫩的雞腿肉，整體吃起來飽嘴但又清爽，一咬下還有水份流出來呢！雞腿排微酸的調味恰到好處'],
  'count': 5},
 'haha': {'content': ['裡面座位不多，還好有跑第一名，不然就沒辦法拍到這畫面啦！這家店最讓我喜歡的，是磚牆上這些畫框和鏡子，畫框裡有的沒有畫，選擇留白，鏡子裡的倒影，真真假假，虛幻難辨',
   '\xa0因為是第一組到的客人，第一位店員對我們十分禮貌殷勤，馬上送上菜單～菜單設計的質感不錯，特別訂製的硬皮',
   '義大利麵最貴不超過＄200，只是選擇不多就是',
   '兒童餐，就是份量小一點，還有用可愛的卡通餐具',
   '這裡給飲料的方式很不一樣，我也是第二次來才搞懂，原來我們點的食尚黑盤早午餐組合─爆料恩力，才有提供紅茶喝到飽，而另一個三明治餐，只有提供一杯，不能續杯',
   '用餐完畢之後還會送上濕紙巾',
   '整體來說這道義大利麵算是普通，可以吃，但說不上美味',
   '經典原味牛肉堡＄200等了算蠻久的，漢堡終於送來了'],
  'count': 8},
 'love': {'content': ['江湖上（？）傳言捲餅很好吃，但我們還沒吃過', '起司也給的十分大方'], 'count': 2},
 'sad': {'content': ['爆料恩力＄200佔了半個盤面的超大歐姆蛋捲，一上桌就帶來震撼！\xa0內容包括時蔬溫沙拉、起司恩力蛋捲、碳烤丹麥布丁土司',
   '沙拉裡面有些

In [11]:
res_json

{'data': [{'ambiguous': False,
   'emotion1': 'wow',
   'emotion2': 'sad',
   'location': 'NA',
   'message': '說起最近高雄地區新開火紅的早午餐店，這間沐濛一定榜上有名，因為餐點份量大、價格實惠，十分受到好評'},
  {'ambiguous': False,
   'emotion1': 'haha',
   'emotion2': 'angry',
   'location': 'NA',
   'message': '裡面座位不多，還好有跑第一名，不然就沒辦法拍到這畫面啦！這家店最讓我喜歡的，是磚牆上這些畫框和鏡子，畫框裡有的沒有畫，選擇留白，鏡子裡的倒影，真真假假，虛幻難辨'},
  {'ambiguous': False,
   'emotion1': 'haha',
   'emotion2': 'wow',
   'location': 'NA',
   'message': '\xa0因為是第一組到的客人，第一位店員對我們十分禮貌殷勤，馬上送上菜單～菜單設計的質感不錯，特別訂製的硬皮'},
  {'ambiguous': False,
   'emotion1': 'angry',
   'emotion2': 'haha',
   'location': 'NA',
   'message': '以下菜單是從 Moment 沐濛 的官方粉絲團借圖來的～（※後來寫文的時候發現，我們現場看到的菜單品項，和臉書上菜單的版本有一點點不一樣，菜色有增減，請以現場店家提供為主，本菜單僅供參考※）點這一頁早午餐組合是最划算的，都有附飲料喝到飽（冰熱紅茶、熱美式咖啡、經典早餐茶）'},
  {'ambiguous': False,
   'emotion1': 'love',
   'emotion2': 'wow',
   'location': 'NA',
   'message': '江湖上（？）傳言捲餅很好吃，但我們還沒吃過'},
  {'ambiguous': False,
   'emotion1': 'angry',
   'emotion2': 'sad',
   'location': 'NA',
   'message': '這一頁蒜辣

In [13]:
total_content[0][0]

8270

## Update Elasticsearch

In [22]:
es.update(index='pixnet', doc_type='food', id=total_content[0][0], body={"doc": {"emotion": emotion_dict}})

{'_id': '8270',
 '_index': 'pixnet',
 '_shards': {'failed': 0, 'successful': 1, 'total': 1},
 '_type': 'food',
 '_version': 18,
 'result': 'updated'}